Leer datos, convertirlos a dataframe, extraer número de factura y nombre del producto y limpiar el nombre del producto para que desaparezca contenido basura para el primer archivo

In [61]:
from dbfread import DBF
import pandas as pd
import re
from mlxtend.preprocessing import TransactionEncoder


archivo_dbf1='sabado/fc06042024.DBF'

tabla_dbf1 = DBF(archivo_dbf1)

df1 = pd.DataFrame(tabla_dbf1)

columnas_deseadas = ['FACNRO', 'DESCRI']
df1 = df1[columnas_deseadas]
df1['DESCRI'] = df1['DESCRI'].apply(lambda x: re.sub(r'\s+\S*$', '', x))

print(df1.head(10))

   FACNRO                     DESCRI
0   61642        LUCKY DAIQUIRI X 10
1   61643            TAPABOCAS X UND
2   61643            DESCONGEL GRIPA
3   61644            AROMATICA  7 OZ
4     581                  EMPANADAS
5   61645         HALLS BARRA  25 GR
6   61646             HALLS X UNIDAD
7   61647          AMERICANO 9 ONZAS
8   61648         AGUA CRISTAL LITRO
9   61648  JET COOKIES AND CREAM 21G


Leer datos, convertirlos a dataframe, extraer número de factura y nombre del producto y limpiar el nombre del producto para que desaparezca contenido basura para el segundo archivo

In [62]:
archivo_dbf2='sabado/fc09032024.DBF'

tabla_dbf2 = DBF(archivo_dbf2)

df2 = pd.DataFrame(tabla_dbf2)

columnas_deseadas = ['FACNRO', 'DESCRI']
df2 = df2[columnas_deseadas]
df2['DESCRI'] = df2['DESCRI'].apply(lambda x: re.sub(r'\s+\S*$', '', x))

print(df2.head(10))

   FACNRO                     DESCRI
0   23102  BRISA LIMON-MANZANA 600ML
1   23102               TIC TAC 16GR
2   49554             VIVE 100 LITRO
3   49554             LUCKY GIN X 10
4   49555        CRISTAL ALOE 330 ML
5   49555                PONKY 33 GR
6   49555                PONKY 33 GR
7   49555      PASTILLAS CHAO CEREZA
8   49556                YOX  100 ML
9   49556   BROWNIE CHOCORRAMO  65GR


Convirtiendo el dataset para que haya una lista de productos por cada número de factura. La información anterior no asocia cuando dos productos hacen parte de la misma factura, que es lo que se hace en ese fragmento. Juntar los dos archivos de información

In [63]:
from collections import defaultdict

facturas = defaultdict(list)

for factura, producto in df1.values:
    facturas[factura].append(producto)

for factura, producto in df2.values:
    facturas[factura].append(producto)

facturas_lista = [[factura, *productos] for factura, productos in facturas.items()]

print(facturas_lista)

[[61642, 'LUCKY DAIQUIRI X 10'], [61643, 'TAPABOCAS X UND', 'DESCONGEL GRIPA'], [61644, 'AROMATICA  7 OZ'], [581, 'EMPANADAS'], [61645, 'HALLS BARRA  25 GR'], [61646, 'HALLS X UNIDAD'], [61647, 'AMERICANO 9 ONZAS'], [61648, 'AGUA CRISTAL LITRO', 'JET COOKIES AND CREAM 21G'], [61649, 'GALLETA NUCITA 20 GR', 'GALLETAS CHOKIS  40 GR'], [61650, 'BRICKET TOKAY'], [61651, 'LUCKY ALASKA X10'], [61652, 'LECHE ALPINA  DESLACTOSADA 200ML'], [61653, 'BOMBAS CROMADA X 12 UN', 'FESTON FELIZ CUMPLEAÑOS METALIZADO'], [61654, 'JUGO HIT  500 ML'], [61655, 'JUMBO MIX  60GR', 'JET COOKIES AND CREAM 21G', 'TRULULU GOMAS/MASMELOS 80GR', 'WAFER JET 22GR'], [61656, 'LUCKY ALASKA X10'], [61657, 'VIVE 100. 380ML', 'EMPANADAS', 'JUMBO 35GR'], [61658, 'LUCKY ALASKA X10'], [61659, 'CLUB SOCIAL   26 GR'], [61660, 'TRULULU UNIDAD'], [61661, 'CLUB SOCIAL   26 GR'], [61662, 'NUTELLA MINI  15GR'], [61663, 'LUCKY GIN X 10'], [61664, 'COCA COLA 3 LITROS'], [61665, 'CHEERS 180GR', 'BOLSA REGALO CRAF MEDIANA'], [61666, 'T

Quitar el número de la factura ya que ya se tienen relacionados los productos de cada transacción

In [65]:
nuevo_dataset = []
for factura in facturas_lista:
    productos = factura[1:]  
    nuevo_dataset.append(productos)

print(nuevo_dataset)


[['LUCKY DAIQUIRI X 10'], ['TAPABOCAS X UND', 'DESCONGEL GRIPA'], ['AROMATICA  7 OZ'], ['EMPANADAS'], ['HALLS BARRA  25 GR'], ['HALLS X UNIDAD'], ['AMERICANO 9 ONZAS'], ['AGUA CRISTAL LITRO', 'JET COOKIES AND CREAM 21G'], ['GALLETA NUCITA 20 GR', 'GALLETAS CHOKIS  40 GR'], ['BRICKET TOKAY'], ['LUCKY ALASKA X10'], ['LECHE ALPINA  DESLACTOSADA 200ML'], ['BOMBAS CROMADA X 12 UN', 'FESTON FELIZ CUMPLEAÑOS METALIZADO'], ['JUGO HIT  500 ML'], ['JUMBO MIX  60GR', 'JET COOKIES AND CREAM 21G', 'TRULULU GOMAS/MASMELOS 80GR', 'WAFER JET 22GR'], ['LUCKY ALASKA X10'], ['VIVE 100. 380ML', 'EMPANADAS', 'JUMBO 35GR'], ['LUCKY ALASKA X10'], ['CLUB SOCIAL   26 GR'], ['TRULULU UNIDAD'], ['CLUB SOCIAL   26 GR'], ['NUTELLA MINI  15GR'], ['LUCKY GIN X 10'], ['COCA COLA 3 LITROS'], ['CHEERS 180GR', 'BOLSA REGALO CRAF MEDIANA'], ['TRIDENT 10 GR'], ['TRULULU CHOCOLORES 30GR', 'TRIDENT 10 GR', 'MENTA CHAO LÍNEA'], ['TRULULU CHOCOLORES 30GR', 'GOL MEGA AREQUIPE 53GR'], ['EMPANADAS', 'ALPIN CAJA  200 ML'], ['PONK

Proceso copiado de la implementación en el ejemplo del profesor ya con en dataset final

In [66]:
te = TransactionEncoder()
te_array = te.fit(nuevo_dataset).transform(nuevo_dataset)
df = pd.DataFrame(te_array, columns=te.columns_)
print(df)

                    DORITO DINAMITA 200 GR   ANTIOQUEÑO VERDE  750 ML  \
0     False                          False                      False   
1     False                          False                      False   
2     False                          False                      False   
3     False                          False                      False   
4     False                          False                      False   
...     ...                            ...                        ...   
8642  False                          False                      False   
8643  False                          False                      False   
8644  False                          False                      False   
8645  False                          False                      False   
8646  False                          False                      False   

       DUX  RELLENAS 36GR   DUX QUESO 36GR  \
0                   False            False   
1                   False      

In [73]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets_ap = apriori(df, min_support=0.01, use_colnames=True)
print(frequent_itemsets_ap)

      support                                 itemsets
0    0.003701          (       DORITO DINAMITA 200 GR)
1    0.001619                    ( DUX  RELLENAS 36GR)
2    0.002776                        ( DUX QUESO 36GR)
3    0.001850    ( KELLOGG'S RICE KRISPIES TREATS 22G)
4    0.004626                  (ACQUA POSTOBON 400 ML)
..        ...                                      ...
498  0.001041   (PASTELES. PALITOS, SANDWICH SENCILLO)
499  0.001156  (TODO RICO BBQ 45GR, PASTELES. PALITOS)
500  0.001041       (PASTELES. PALITOS, TRIDENT 10 GR)
501  0.001156     (PASTILLAS CHAO CEREZA, PONKY 33 GR)
502  0.001503   (PASTILLAS CHAO CEREZA, TRIDENT 10 GR)

[503 rows x 2 columns]


In [69]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets_fp=fpgrowth(df, min_support=0.01, use_colnames=True)
print(frequent_itemsets_fp)

     support                       itemsets
0   0.107089                    (EMPANADAS)
1   0.022898           (HALLS BARRA  25 GR)
2   0.015612             (LUCKY ALASKA X10)
3   0.036660             (JUGO HIT  500 ML)
4   0.019544              (VIVE 100. 380ML)
5   0.027871                (TRIDENT 10 GR)
6   0.021279             (MENTA CHAO LÍNEA)
7   0.014456      (TRULULU CHOCOLORES 30GR)
8   0.011102           (ALPIN CAJA  200 ML)
9   0.019313                  (PONKY 33 GR)
10  0.027524        (PASTILLAS CHAO CEREZA)
11  0.013299    (BRISA LIMON-MANZANA 600ML)
12  0.011680            (PONY MALTA 330 ML)
13  0.021742            (GATORADE   500 ML)
14  0.011102             (SPEED MAX  269ML)
15  0.011449           (TODO RICO BBQ 45GR)
16  0.010640              (FUZE TEA  400ML)
17  0.044524            (PASTELES. PALITOS)
18  0.037932       (COCA COLA FLEXI 400 ML)
19  0.030531              (CANDYRANCH 13GR)
20  0.013184               (MONSTER 473 ML)
21  0.013068            (ROTHMAN

In [77]:
from mlxtend.frequent_patterns import association_rules

rules_ap = association_rules(frequent_itemsets_ap, metric="confidence", min_threshold=0.8)

rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.8)

In [78]:
print(rules_ap)

       antecedents  consequents  antecedent support  consequent support  \
0  (MOCACHINO 7OZ)  (EMPANADAS)            0.002082            0.107089   

    support  confidence      lift  leverage  conviction  zhangs_metric  
0  0.001156    0.555556  5.187785  0.000934    2.009049       0.808923  


In [76]:
print(rules_fp)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
